In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

all_bus_details = []  # List to hold all bus details

In [2]:
def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

In [3]:
def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

In [4]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

In [5]:
def scrape_bus_details(driver, url, route_name, state):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load

        states={"astc":"Assam","tnstc":"Tamil Nadu","upsrtc":"Uttar Pradesh","tsrtc":"Telengana","rsrtc":"Rajastan","gsrtc":"Gujarat",
                "apsrtc":"Andhra Pradesh","jksrtc":"Jammu & Kashmir","hrtc":"Haryana","south-bengal-state-transport-corporation-sbstc":"West Bengal",
                "puducherry-road-transport-corporation-prtc":"Pondicherry","ksrtc-karnataka":"Karnataka","ksrtc-kerala":"Kerela"}      
 
        # Extract state from the URL if it's part of the URL
        #state = url.split("/")[-1]  # Example: Extracts the last part of the URL after '/'
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")
            
            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text.replace("INR ", ""),
                    "Seat_Availability": (''.join(filter(str.isdigit, seat_availability_elements[i].text)) if i < len(seat_availability_elements) 
                                          else '0'),
                    "State": states.get(state, "State not found")  # Adding state here
                }
                bus_details.append(bus_detail)
                
            return bus_details
        
        except Exception as e:
            print("No buses found")
            return []
    
    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

In [6]:
# Function to navigate through pages
def navigate_through_pages(URL):
    """Navigate through pages and scrape bus routes and details."""
    global all_bus_details

    try:
        driver = initialize_driver()
        load_page(driver, URL)
        state = URL.split("/")[-1]
        # Find total number of pages
        try:
            page_tabs = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, "DC_117_pageTabs")))
            num_pages = len(page_tabs)
            print(f"Total pages found: {num_pages}")
        except Exception as e:
            print("Bus Route terimnation")
            num_pages = 1
        
        # Loop through each page
        for page in range(1, num_pages + 1):
            try:
                driver = initialize_driver()
                load_page(driver, URL)
                print(f"Scraping page {page}...")

                # Handle pagination for pages > 1
                if page > 1:
                    # Re-locate pagination element to avoid stale element reference
                    pagination_tab = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable(
                            (By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs') and text()='{page}']")
                        )
                    )
                    driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                    driver.execute_script("arguments[0].click();", pagination_tab)

                    # Wait for routes to load
                    WebDriverWait(driver, 10).until(
                        EC.presence_of_all_elements_located((By.CLASS_NAME, "route"))
                    )

                # Scrape routes from the current page
                bus_routes_link, bus_routes_name = scrape_bus_routes(driver)

                # Scrape details for each route
                for link, name in zip(bus_routes_link, bus_routes_name):
                    print(f"Scraping route: {name}")
                    bus_details = scrape_bus_details(driver, link, name, state)
                    if bus_details:
                        all_bus_details.extend(bus_details)

            except Exception as e:
                print(f"Error navigating to page {page}: {e}")

    except Exception as e:
        print(f"Error occurred while navigating pages: {e}")

    finally:
        driver.quit()

In [7]:
def main():
    # Scrape routes and details from all pages
    url = ['https://www.redbus.in/online-booking/astc', 'https://www.redbus.in/online-booking/tnstc', 
         'https://www.redbus.in/online-booking/upsrtc', 'https://www.redbus.in/online-booking/tsrtc', 
         'https://www.redbus.in/online-booking/rsrtc', 'https://www.redbus.in/online-booking/gsrtc', 
         'https://www.redbus.in/online-booking/apsrtc', 'https://www.redbus.in/online-booking/jksrtc', 
         'https://www.redbus.in/online-booking/hrtc', 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc',
         'https://www.redbus.in/online-booking/puducherry-road-transport-corporation-prtc', 'https://www.redbus.in/online-booking/ksrtc-karnataka', 
         'https://www.redbus.in/online-booking/ksrtc-kerala']

    for ite in url:
        navigate_through_pages(ite)
        # Convert the list of dictionaries to a DataFrame
        df = pd.DataFrame(all_bus_details)
        # Save the DataFrame to a CSV file
        df.to_csv('bus_details.csv', index=False)
    

In [8]:
if __name__ == "__main__":
    main()

Total pages found: 5
Scraping page 1...
Scraping route: Tezpur to Guwahati
Scraping route: Guwahati to Tezpur
Scraping route: Nagaon (Assam) to Guwahati
Scraping route: Guwahati to Nagaon (Assam)
Scraping route: Goalpara to Guwahati
Scraping route: Jorhat to North Lakhimpur
No buses found
Scraping route: North Lakhimpur to Sibsagar (Assam)
Scraping route: Guwahati to Dhubri
Scraping route: Dhubri to Guwahati
Scraping route: Dhekiajuli to Guwahati
No buses found
Scraping page 2...
Scraping route: Sibsagar (Assam) to North Lakhimpur
No buses found
Scraping route: North Lakhimpur to Jorhat
Scraping route: Jorhat to Dibrugarh
Scraping route: Jorhat to Dhemaji
No buses found
Scraping route: Jorhat to Tinsukia
Scraping route: Dhemaji to Jorhat
No buses found
Scraping route: Guwahati to Biswanath Charali
Scraping route: Tezpur to Dibrugarh
Scraping route: North Lakhimpur to Dibrugarh
Scraping route: North Lakhimpur to Tezpur
Scraping page 3...
Scraping route: Dibrugarh to Jorhat
Scraping rout